# <font color="#5fa8d3"> Yolo model


## <font color="#62b6cb"> Table of Content <a name="ToC"></a>
0. [Libraries Importation, Parameters & Funtions Definition](#id0)<br>
1. [Preparation of the data](#id1)<br>
    1.1 [Aply transformations](#id11)<br>
    1.2 [Join new data to train set](#id12)<br>
2. [Prices](#id2)<br>

## <font color="#62b6cb"> 0. Libraries Importation, Parameters & Funtions Definition <a name="id0"></a>

In [2]:
import json
import os
from PIL import Image
from ultralytics import YOLO
import shutil
import random
import cv2
import matplotlib.pyplot as plt
from ultralytics.data.converter import convert_coco
import yaml # for importing a yaml file
from src.mymodule import * # for importing our functions

ModuleNotFoundError: No module named 'ultralytics'

**Configuration Variables**

In [ ]:
# Load the YAML file
with open('..\config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Function to set variables globally and store their names
def set_variables(config, prefix='', var_dict={}):
    for key, value in config.items():
        if isinstance(value, dict):
            set_variables(value, prefix + key + '_', var_dict)
        else:
            globals()[prefix + key] = value
            var_dict[prefix + key] = value
    return var_dict

# Set variables globally and get a dictionary of the set variables
set_vars = set_variables(config)

# Print all the variables that were set
print("Variables set from YAML file:")
for var_name, var_value in set_vars.items():
    print(f"{var_name}: {var_value}")

In [1]:
# PARAMETERS
# paths
train_annotations_json = "annotations.json"
val_annotations_json = "annotations.json"
test_annotations_json = "annotations.json"
path_split = "./CustomImages/" # folder with sets of train, val and test
folder_train_split = "train"
folder_val_split = "val"
folder_test_split = "test"
path_train_folder_split = os.path.join(path_split, folder_train_split)
path_val_folder_split = os.path.join(path_split, folder_val_split)
path_test_folder_split = os.path.join(path_split, folder_test_split)
folder_yolo_train = "labels"
folder_yolo_val = "labels"
weights_path = './yolo_weights'

NameError: name 'os' is not defined

In [3]:
def validate_coco_dataset(coco_annotation_path, images_folder_path):
    """
    Validates a COCO dataset to check if the dimensions, IDs, and naming conventions are correct.

    Parameters:
    coco_annotation_path (str): Path to the COCO annotations JSON file.
    images_folder_path (str): Path to the folder containing images.

    Returns:
    bool: True if the dataset is valid, False otherwise.
    """
    # Load the COCO annotations
    with open(coco_annotation_path, 'r') as file:
        coco_data = json.load(file)

    # Create a set to store unique image IDs
    image_ids = set()

    # Validate images
    for image_info in coco_data['images']:
        image_id = image_info['id']
        file_name = image_info['file_name']
        width = image_info['width']
        height = image_info['height']

        # Check if image ID is unique
        if image_id in image_ids:
            print(f"Duplicate image ID found: {image_id}")
            return False
        image_ids.add(image_id)

        # Check if the image file exists
        image_path = os.path.join(os.path.join(images_folder_path), file_name)
        if not os.path.exists(image_path):
            print(f"Image file not found: {file_name}")
            return False

        # Check if the image dimensions match
        with Image.open(image_path) as img:
            if img.width != width or img.height != height:
                print(f"Image dimensions do not match for {file_name}: "
                      f"expected ({width}, {height}), got ({img.width}, {img.height})")
                return False

    # Validate annotations
    for annotation in coco_data['annotations']:
        image_id = annotation['image_id']
        if image_id not in image_ids:
            print(f"Annotation references non-existent image ID: {image_id}")
            return False
    return True

In [4]:
def clamp(value, min_value, max_value):
    """
    Ensures a value is within a specified range.

    Args:
        value (float): The value to adjust.
        min_value (float): The minimum allowable value.
        max_value (float): The maximum allowable value.

    Returns:
        float: The adjusted value within the specified range.
    """
    return max(min(value, max_value), min_value)

def adjust_polygon(polygon, image_width, image_height):
    """
    Adjusts the points of a polygon to ensure they stay within the image boundaries.

    Args:
        polygon (list): List of polygon coordinates in the format [x1, y1, x2, y2, ..., xn, yn].
        image_width (int): The width of the image.
        image_height (int): The height of the image.

    Returns:
        list: The adjusted polygon coordinates.
    """
    adjusted_polygon = []
    for i in range(0, len(polygon), 2):
        x = clamp(polygon[i], 0, image_width - 1)
        y = clamp(polygon[i+1], 0, image_height - 1)
        adjusted_polygon.extend([x, y])
    return adjusted_polygon

def process_coco_annotations(coco_annotations_path, output_path):
    """
    Processes COCO annotations to adjust polygons to stay within image boundaries.

    Args:
        coco_annotations_path (str): The file path to the input COCO annotations JSON file.
        output_path (str): The file path to save the adjusted COCO annotations JSON file.
    """
    with open(coco_annotations_path, 'r') as f:
        coco_data = json.load(f)
    
    for annotation in coco_data['annotations']:
        segmentation = annotation['segmentation']
        for i, polygon in enumerate(segmentation):
            # Assuming the segmentation is in the format [x1, y1, x2, y2, ..., xn, yn]
            if len(polygon) % 2 != 0:
                print(f"Warning: Polygon with odd number of coordinates found in annotation ID {annotation['id']}")
                continue
            
            image_id = annotation['image_id']
            image_info = next(img for img in coco_data['images'] if img['id'] == image_id)
            image_width = image_info['width']
            image_height = image_info['height']
            
            segmentation[i] = adjust_polygon(polygon, image_width, image_height)
    
    with open(output_path, 'w') as f:
        json.dump(coco_data, f, indent=4)

In [5]:
def convert_coco_to_yolo_segmentation(coco_json_path, output_dir):
    """
    Converts COCO segmentation annotations to YOLO format.

    Args:
    coco_json_path (str): Path to the COCO annotations JSON file.
    output_dir (str): Path to the folder to save YOLO annotations.
    """
    # Load COCO JSON
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Create a directory for YOLO annotations
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create a mapping from COCO category IDs to YOLO class IDs (1-9 to 0-8)
    category_mapping = {category['id']: category['id'] - 1 for category in coco_data['categories']}

    # Iterate over each image in the COCO dataset
    for image_info in coco_data['images']:
        image_id = image_info['id']
        image_width = image_info['width']
        image_height = image_info['height']
        image_filename = image_info['file_name']

        # Prepare the YOLO annotation filename
        yolo_annotation_filename = os.path.splitext(image_filename)[0] + ".txt"  # separate from .jpg
        yolo_annotation_path = os.path.join(output_dir, yolo_annotation_filename)

        # Open the YOLO annotation file for writing
        with open(yolo_annotation_path, 'w') as yolo_file:
            # Iterate over each annotation
            for annotation in coco_data['annotations']:
                if annotation['image_id'] == image_id:
                    category_id = annotation['category_id']
                    segmentation = annotation['segmentation']

                    # Check if the annotation has polygon segmentation
                    if isinstance(segmentation, list):
                        for polygon in segmentation:
                            # Normalize the coordinates by the dimensions of the image
                            normalized_polygon = [(x / image_width, y / image_height) for x, y in zip(polygon[0::2], polygon[1::2])] 
                             # x: even indices, y: odd indices

                            # Flatten the normalized coordinates
                            normalized_polygon_str = ' '.join([f"{x} {y}" for x, y in normalized_polygon])

                            # Get the YOLO class ID
                            yolo_class_id = category_mapping[category_id]

                            # Write the annotation in YOLO format
                            yolo_file.write(f"{yolo_class_id} {normalized_polygon_str}\n")

In [3]:
def copy_or_move_coco_images(coco_annotation_file, source_images_folder, destination_images_folder, operation="copy"):
    """
    Copy or move images specified in a COCO annotation file from a source folder to a destination folder.

    This function reads a COCO (Common Objects in Context) annotation file to get the list of image file names
    and either copies or moves those images from the source folder to the destination folder based on the operation
    parameter. If the destination folder does not exist, it will be created.

    Parameters:
    coco_annotation_file (str): Path to the COCO annotation JSON file.
    source_images_folder (str): Path to the source folder containing the images.
    destination_images_folder (str): Path to the destination folder where images will be copied or moved.
    operation (str): Operation to perform: "copy" or "move". Default is "copy".

    Returns:
    None

    Example:
    copy_or_move_coco_images(
        coco_annotation_file='annotations/instances_train2017.json',
        source_images_folder='images/train2017',
        destination_images_folder='images/processed_train2017',
        operation='move'
    )

    Notes:
    - The function checks if each image exists in the source folder before copying or moving it.
    - If an image is not found in the source folder, a message is printed.
    - The function uses `shutil.copy` to copy the images and `shutil.move` to move the images.
    """
    # Create destination folder if it does not exist
    os.makedirs(destination_images_folder, exist_ok=True)

    # Load the COCO annotation file
    with open(coco_annotation_file, 'r') as f:
        coco_data = json.load(f)

    # Create a dictionary to store image_id to file_name mapping
    image_id_to_filename = {image['id']: image['file_name'] for image in coco_data['images']}

    # Process each image
    for image_id, filename in image_id_to_filename.items():
        source_image_path = os.path.join(source_images_folder, filename)
        destination_image_path = os.path.join(destination_images_folder, filename)

        # Copy or move image to destination folder
        if os.path.exists(source_image_path):
            if operation == "copy":
                shutil.copy(source_image_path, destination_image_path)
                print(f"Copied: {source_image_path} to {destination_image_path}")
            elif operation == "move":
                shutil.move(source_image_path, destination_image_path)
                print(f"Moved: {source_image_path} to {destination_image_path}")
            else:
                raise ValueError("Operation must be 'copy' or 'move'.")
        else:
            print(f"Image {filename} not found in source folder.")

In [ ]:
def move_and_rename_folder(source_folder, destination_root, new_folder_name):
    """
        Move and rename a folder

        Args:
            source_folder: path to the folder to move
            destination_root: new path
            new_folder_name: new name of the folder

    """
    destination_folder = os.path.join(destination_root, new_folder_name)
    shutil.move(source_folder, destination_folder)



In [4]:
def create_yaml_file(file_path, train_path, val_path, nc, names):
    """
    Create yaml for yolo.

        Args:
            file_path: path of the yaml
            train_path: path of the train set
            val_path: path of the val set
            nc: number of categories
            names: names of the categories
    """
    data = {
        'train': train_path,
        'val': val_path,
        'nc': nc,
        'names': names
    }

    with open(file_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

## <font color="#62b6cb"> 1. Preparation of data <a name="id1"></a>

Check the annotations and id of the coco jsons

In [8]:
print("Check that annotation for training is correct")
print(validate_coco_dataset(os.path.join(path_train_folder_split,train_annotations_json), os.path.join(path_train_folder_split, 'images')))
print("Check that annotation for validation is correct")
print(validate_coco_dataset(os.path.join(path_val_folder_split,val_annotations_json), os.path.join(path_val_folder_split, 'images')))
print("Check that annotation for test is correct")
print(validate_coco_dataset(os.path.join(path_test_folder_split,test_annotations_json), path_test_folder_split))

Check that annotation for training is correct
True
Check that annotation for validation is correct
True
Check that annotation for test is correct
True


Some of the points of the polygons are out of the range of the width and the height

In [29]:
# train
process_coco_annotations(os.path.join(path_train_folder_split,train_annotations_json),
                          os.path.join(path_train_folder_split,train_annotations_json))

# val
process_coco_annotations(os.path.join(path_val_folder_split, val_annotations_json), 
                         os.path.join(path_val_folder_split,val_annotations_json))

Obtain the yolo txt for each images from the coco data annotations

In [9]:
convert_coco(labels_dir= path_train_folder_split,save_dir= './CustomImages/aux/', use_segments=True)
convert_coco(labels_dir= path_val_folder_split,save_dir= './CustomImages/aux2/', use_segments=True)

In [ ]:
move_and_rename_folder("./CustomImages/aux/labels/annotations", path_train_folder_split,
                       "labels" )

shutil.rmtree('./CustomImages/aux')

move_and_rename_folder("./CustomImages/aux2/labels/annotations", path_val_folder_split,
                       "labels" )
shutil.rmtree('./CustomImages/aux2')

Move the images to the folder 'images' to prepare eveything for the yolo model.


In [14]:
copy_or_move_coco_images(
    coco_annotation_file=os.path.join(path_val_folder_split, val_annotations_json),
    source_images_folder=path_val_folder_split,
    destination_images_folder=os.path.join(path_val_folder_split, 'images'),
    operation='move'
)

In [13]:
def select_images(source_folder, destination_folder, num_images):
    images_folder = os.path.join(source_folder, 'images')
    labels_folder = os.path.join(source_folder, 'labels')

    if not os.path.exists(destination_folder):
        os.makedirs(os.path.join(destination_folder, 'images'))
        os.makedirs(os.path.join(destination_folder, 'labels'))

    images = os.listdir(images_folder)
    selected_images = random.sample(images, num_images)

    for image in selected_images:
        image_name = os.path.splitext(image)[0]
        source_image_path = os.path.join(images_folder, image)
        destination_image_path = os.path.join(destination_folder, 'images', image)
        
        source_label_path = os.path.join(labels_folder, f"{image_name}.txt")
        destination_label_path = os.path.join(destination_folder, 'labels', f"{image_name}.txt")
        
        shutil.copy(source_image_path, destination_image_path)
        if os.path.exists(source_label_path):
            shutil.copy(source_label_path, destination_label_path)

# Folder configuration
train_folder = './CustomImages/train/'
val_folder = './CustomImages/val/'

destination_train_folder = './CustomImages/train_prueba'
destination_val_folder = './CustomImages/val_prueba'

# Select 100 images from each folder
select_images(train_folder, destination_train_folder, 100)
select_images(val_folder, destination_val_folder, 100)


In [14]:
with open(os.path.join(path_train_folder_split, train_annotations_json), 'r') as f:
    coco_data = json.load(f)

names = [class_name['name'] for class_name in coco_data["categories"]]

# Specify the paths and information
actual_path = os.getcwd()
train_path = os.path.join(actual_path, 'CustomImages/train_prueba/images')
val_path = os.path.join(actual_path, 'CustomImages/val_prueba/images')

names_categories = [class_name['name'] for class_name in coco_data["categories"]]
nc = len(names)
file_path = 'data.yaml'

# Create the YAML file
create_yaml_file(file_path, train_path, val_path, nc, names)


NameError: name 'create_yaml_file' is not defined

In [36]:
# %reset  # eliminate variables defined in the session

Don't know how to reset  #, please run `%reset?` for details
Don't know how to reset  eliminate, please run `%reset?` for details
Don't know how to reset  variables, please run `%reset?` for details
Don't know how to reset  defined, please run `%reset?` for details
Flushing input history
Don't know how to reset  the, please run `%reset?` for details
Don't know how to reset  session, please run `%reset?` for details


In [17]:
if not os.path.exists(weights_path):
    os.makedirs(weights_path)
model = YOLO(os.path.join(weights_path, "yolov8m-seg.pt"))

In [18]:
model.train(data="data.yaml", epochs=3, optimizer='Adam', task='segment')

New https://pypi.org/project/ultralytics/8.2.63 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.62  Python-3.12.4 torch-2.3.1+cu118 CPU (12th Gen Intel Core(TM) i7-1255U)
engine\trainer: task=segment, mode=train, model=./yolo_weights\yolov8m-seg.pt, data=data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

train: Scanning C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\CustomImages\train_prueba\labels... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 618.31it/s]

train: New cache created: C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\CustomImages\train_prueba\labels.cache



val: Scanning C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\CustomImages\val_prueba\labels... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 1331.52it/s]

val: New cache created: C:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\CustomImages\val_prueba\labels.cache


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  50%|█████     | 2/4 [00:58<00:57, 28.98s/it]

WARNING  NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [01:40<00:00, 25.12s/it]


                   all        100        224      0.558     0.0149   0.000852   0.000118      0.444    0.00126   6.68e-06   1.34e-06
              mis_lost         11         13          1          0          0          0          1          0          0          0
              met_tear         29         40          1          0          0          0          0          0          0          0
           met_scratch         42         88   0.000503     0.0227   0.000126   6.22e-05   0.000235     0.0114   6.01e-05    1.2e-05
           glass_crack         18         18      0.018      0.111    0.00754   0.000996          0          0          0          0
             mis_punct         15         21          1          0          0          0          1          0          0          0
              mis_lamp         13         14          1          0          0          0          1          0          0          0
        met_dent_minor          9         10          0          0   

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001BD0513D790>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039

In [19]:
results = model.predict("03bda226ad62553c0c73.jpg")


image 1/1 c:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\03bda226ad62553c0c73.jpg: 480x640 300 met_scratchs, 501.6ms
Speed: 5.4ms preprocess, 501.6ms inference, 145.0ms postprocess per image at shape (1, 3, 480, 640)


In [34]:
results[0].masks

ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,

In [22]:
# Realizar predicciones
results = model.predict("03bda226ad62553c0c73.jpg", conf=0.9, iou=0.7)

# Visualizar resultados
for result in results:
    img = result.plot(show=True, labels=True, masks=True, boxes=True)
    plt.imshow(img)
    plt.show()


image 1/1 c:\Users\teres\OneDrive\Documentos\UCD\Summer\project-danielteresa\03bda226ad62553c0c73.jpg: 480x640 132 met_scratchs, 412.3ms
Speed: 0.0ms preprocess, 412.3ms inference, 58.0ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 640x480 with 1 Axes>